# Data Cleaner for Delighted Scores
This app will clean up raw data that's exported from Delighted, in preparation for importing it into Accelo.  The specific steps which it will complete are:
- Remove all columns except for:
-- Name
-- Email
-- Score
-- Comment
-- Response Timestamp
-- Access Level
-- Company Name
-- Company ID
-- Delighted Country
-- Domain
-- Industry
-- Manager
-- MRR
- Change "Score" column's "0" rows to "Zero"
- Copy "Score" column's content into new column, and rename it to "Title"
- Add empty company_id values
- Format the "Response Timestamp" column as a date
- Split "Name" column into multiple columns by space

In [279]:
# Import library dependencies
import pandas as pd
import numpy as np
import requests
import pprint as pp
import math
import re
import base64

In [281]:
# API Variables
deployment = 'markjellison'
token = 'e0lsIgICJ2rDoxzfFYGEJlgXiA~bfR9y'

In [282]:
# Query Defaults
email = 'customer.owner.affinitylive@gmail.com'
companyId= '2195'

In [283]:
# API Endpoint Setup
urlAffiliations = f'https://{deployment}.api.accelo.com/api/v0/affiliations?_fields=id,email,company&_search={email}&_limit=1'
urlCompanies = f'https://{deployment}.api.accelo.com/api/v0/companies?_filters=id({companyId})&_fields=name&_limit=1'

In [284]:
# API Header Setup
headers = {'Content-Type': 'application/json',
          'Authorization': f'Bearer {token}'}

In [302]:
try:
    # Open the source CSV
    source = pd.read_csv('CSV Files/delighted-data_24-February-2020.csv')
    source.head()
except:
    print("Bad filename")

In [299]:
# Open the source CSV
source = pd.read_csv('CSV Files/delighted-data_24-February-2020.csv')
source.head()

,Response ID,Name,Email,Score,Asset Title,Comment,Response Timestamp,Notes,Tags,access_level,...,financial_visibility,industry_template,manager,monthly_spend,pro_users,product,projects,psa,retainers,service
0,115439005,Kenneth Sejberg,kse@simplesolution.dk,0,0,NaN,2/24/2020 2:56,NaN,NaN,professional,...,NaN,it-consulting,Marie Barker,2070.0,30,psa,premium,NaN,premium,premium
1,115435153,Valentina Schichl,valentina.schichl@kaiser-x.com,2,0,NaN,2/24/2020 1:19,NaN,NaN,professional,...,NaN,it-consulting,Marie Barker,4182.0,95,mixed,premium,NaN,NaN,NaN
2,115431073,Velton Davis,velton@prospergroupcorp.com,3,0,"Very slow, convoluted, several errors.",2/23/2020 23:36,NaN,NaN,professional,...,NaN,digital-marketing,Marie Barker,2530.0,48,psa,premium,NaN,premium,premium
3,115393412,Francesca McMaster,fmcmaster@extent.com.au,0,0,NaN,2/23/2020 14:14,NaN,NaN,professional,...,NaN,other,Amy Stewart,2450.0,53,mixed,premium,NaN,NaN,NaN
4,115393239,Plamen | Verve?,plamen@verve-design.co.uk,0,0,NaN,2/23/2020 14:12,NaN,NaN,professional,...,NaN,web-design,Clay Hagen,759.0,10,psa,premium,NaN,premium,premium


In [286]:
# Create a new DF with just the relevant columns
data = source[['Name', 'Email', 'Score', 'Comment', 'Response Timestamp', 'access_level', 'company_id', 'company_name', 'Delighted Country', 'domain', 'industry_template', 'manager', 'monthly_spend']]
data.head()

,Name,Email,Score,Comment,Response Timestamp,access_level,company_id,company_name,Delighted Country,domain,industry_template,manager,monthly_spend
0,Kenneth Sejberg,kse@simplesolution.dk,0,NaN,2/24/2020 2:56,professional,194190.0,Simple Solution,Denmark,simplesolution.accelo.com,it-consulting,Marie Barker,2070.0
1,Valentina Schichl,valentina.schichl@kaiser-x.com,2,NaN,2/24/2020 1:19,professional,199938.0,Kaiser X Labs,Germany,kaiserxlabs.accelo.com,it-consulting,Marie Barker,4182.0
2,Velton Davis,velton@prospergroupcorp.com,3,"Very slow, convoluted, several errors.",2/23/2020 23:36,professional,175217.0,The Prosper Group,United States,prosper-group-corp.accelo.com,digital-marketing,Marie Barker,2530.0
3,Francesca McMaster,fmcmaster@extent.com.au,0,NaN,2/23/2020 14:14,professional,180972.0,Extent Heritage,Australia,extent.accelo.com,other,Amy Stewart,2450.0
4,Plamen | Verve?,plamen@verve-design.co.uk,0,NaN,2/23/2020 14:12,professional,152975.0,Verve Graphic Design & Marketing,United Kingdom,verve.accelo.com,web-design,Clay Hagen,759.0


In [287]:
# Rename the columns
data = data.rename(columns={"access_level": "Access Level", "company_id": "Company ID", "company_name": "Company Name", "domain": "Domain", "industry_template": "Industry", "manager": "Manager", "monthly_spend": "MRR", "Name": "Original Name"})
data.head()

,Original Name,Email,Score,Comment,Response Timestamp,Access Level,Company ID,Company Name,Delighted Country,Domain,Industry,Manager,MRR
0,Kenneth Sejberg,kse@simplesolution.dk,0,NaN,2/24/2020 2:56,professional,194190.0,Simple Solution,Denmark,simplesolution.accelo.com,it-consulting,Marie Barker,2070.0
1,Valentina Schichl,valentina.schichl@kaiser-x.com,2,NaN,2/24/2020 1:19,professional,199938.0,Kaiser X Labs,Germany,kaiserxlabs.accelo.com,it-consulting,Marie Barker,4182.0
2,Velton Davis,velton@prospergroupcorp.com,3,"Very slow, convoluted, several errors.",2/23/2020 23:36,professional,175217.0,The Prosper Group,United States,prosper-group-corp.accelo.com,digital-marketing,Marie Barker,2530.0
3,Francesca McMaster,fmcmaster@extent.com.au,0,NaN,2/23/2020 14:14,professional,180972.0,Extent Heritage,Australia,extent.accelo.com,other,Amy Stewart,2450.0
4,Plamen | Verve?,plamen@verve-design.co.uk,0,NaN,2/23/2020 14:12,professional,152975.0,Verve Graphic Design & Marketing,United Kingdom,verve.accelo.com,web-design,Clay Hagen,759.0


In [288]:
data.loc[(data.Score == 0), 'Score']='Zero'
data.head()

,Original Name,Email,Score,Comment,Response Timestamp,Access Level,Company ID,Company Name,Delighted Country,Domain,Industry,Manager,MRR
0,Kenneth Sejberg,kse@simplesolution.dk,Zero,NaN,2/24/2020 2:56,professional,194190.0,Simple Solution,Denmark,simplesolution.accelo.com,it-consulting,Marie Barker,2070.0
1,Valentina Schichl,valentina.schichl@kaiser-x.com,2,NaN,2/24/2020 1:19,professional,199938.0,Kaiser X Labs,Germany,kaiserxlabs.accelo.com,it-consulting,Marie Barker,4182.0
2,Velton Davis,velton@prospergroupcorp.com,3,"Very slow, convoluted, several errors.",2/23/2020 23:36,professional,175217.0,The Prosper Group,United States,prosper-group-corp.accelo.com,digital-marketing,Marie Barker,2530.0
3,Francesca McMaster,fmcmaster@extent.com.au,Zero,NaN,2/23/2020 14:14,professional,180972.0,Extent Heritage,Australia,extent.accelo.com,other,Amy Stewart,2450.0
4,Plamen | Verve?,plamen@verve-design.co.uk,Zero,NaN,2/23/2020 14:12,professional,152975.0,Verve Graphic Design & Marketing,United Kingdom,verve.accelo.com,web-design,Clay Hagen,759.0


In [289]:
# Copy the contents of the Score column as "Title"
data['Title'] = data['Score']
data.head()

,Original Name,Email,Score,Comment,Response Timestamp,Access Level,Company ID,Company Name,Delighted Country,Domain,Industry,Manager,MRR,Title
0,Kenneth Sejberg,kse@simplesolution.dk,Zero,NaN,2/24/2020 2:56,professional,194190.0,Simple Solution,Denmark,simplesolution.accelo.com,it-consulting,Marie Barker,2070.0,Zero
1,Valentina Schichl,valentina.schichl@kaiser-x.com,2,NaN,2/24/2020 1:19,professional,199938.0,Kaiser X Labs,Germany,kaiserxlabs.accelo.com,it-consulting,Marie Barker,4182.0,2
2,Velton Davis,velton@prospergroupcorp.com,3,"Very slow, convoluted, several errors.",2/23/2020 23:36,professional,175217.0,The Prosper Group,United States,prosper-group-corp.accelo.com,digital-marketing,Marie Barker,2530.0,3
3,Francesca McMaster,fmcmaster@extent.com.au,Zero,NaN,2/23/2020 14:14,professional,180972.0,Extent Heritage,Australia,extent.accelo.com,other,Amy Stewart,2450.0,Zero
4,Plamen | Verve?,plamen@verve-design.co.uk,Zero,NaN,2/23/2020 14:12,professional,152975.0,Verve Graphic Design & Marketing,United Kingdom,verve.accelo.com,web-design,Clay Hagen,759.0,Zero


In [290]:
for index, row in data.iterrows():
    companyName = row['Company Name']
    splitString = row['Response Timestamp'].split()
    data.loc[(data['Company Name'] == companyName), 'Response Timestamp']=splitString[0]
    
    try:
        if math.isnan(row['Company ID']):
            email = row['Email']
            urlAffiliations = f'https://{deployment}.api.accelo.com/api/v0/affiliations?_fields=id,email,company&_search={email}&_limit=1'
            try:
                response = requests.get(urlAffiliations, headers=headers).json()
                if response['response'] != []:

                    companyId = response['response'][0]['company']
                    data.loc[(data['Company Name'] == companyName), 'Company ID']=companyId                
            except requests.exceptions.RequestException as e:
                print(f'Error: {e}')
    except:
        print("Already processed the dataframe")
print("Processing complete.")

Processing complete.


In [292]:
# Clean up the respondents' names
names = data['Original Name'].tolist()
updatedNames = []
for name in range(0, len(names)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(names[name]))
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    updatedNames.append(document)
updatedNames

['Kenneth Sejberg',
 'Valentina Schichl',
 'Velton Davis',
 'Francesca McMaster ',
 'Plamen Verve ',
 'Josh Marks',
 'Chris Bennett',
 'Sara Moser',
 'Jessica Owens',
 'Mike Johnson',
 'Ben Bryant',
 'Spencer Jenkins',
 'Patrick Gatewood',
 'Todd Huffine',
 'Charles Stimec',
 'Jodie Goble',
 'Esin Ustundag',
 'Kareem Schwarz',
 'Tom Dawson',
 'Ashwin Charles',
 'James Mumford',
 'Brandon Greiner',
 'Gerald Smith',
 'Dan Sirower',
 'Tomasz Buklaho',
 'Matt Douglas',
 'Gavin Steinberg',
 'Matthew Scholten',
 'Samuel Tomkins',
 'Salvador Fernandez Tejedo',
 'Monica Popplestone',
 'Bobi Gavrovski',
 'Shaun Stanbridge',
 'Paul Bowen',
 'Anthony Ranucci',
 'Shayne Annett',
 'Tim Dodds',
 'Ellie Bakker',
 'Trae Dantzler',
 'Austin Fisk',
 'Lucy Arnold',
 'Marc Andr Dubois',
 'James Brown',
 'Hiten Shah',
 'Bernard Mangelsdorf',
 'Lee Anne Molony',
 'Chris Haines',
 'Sarah Krup',
 'Pierre Bourassa',
 'Joe Hoani',
 'Artwell Nyamugumura',
 'Christopher Sedlak',
 'Rex Smith',
 'Print_1 Mick',
 'J

In [293]:
nameLength = []
name1 = []
name2 = []
name3 = []
name4 = []
name5 = []
for i in updatedNames:
    splitName = i.split()
    name1.append(splitName[0])
    try:
        name2.append(splitName[1])
    except:
        name2.append('')
    try:
        name3.append(splitName[2])
    except:
        name3.append('')
    try:
        name4.append(splitName[3])
    except:
        name4.append('')
    try:
        name5.append(splitName[4])
    except:
        name5.append('')
data['First Name'] = name1
data['Last Name'] = name2
data['Other Name 1'] = name3
data['Other Name 2'] = name4
data['Other Name 3'] = name5

In [294]:
data.head()

,Original Name,Email,Score,Comment,Response Timestamp,Access Level,Company ID,Company Name,Delighted Country,Domain,Industry,Manager,MRR,Title,First Name,Last Name,Other Name 1,Other Name 2,Other Name 3
0,Kenneth Sejberg,kse@simplesolution.dk,Zero,NaN,2/24/2020,professional,194190,Simple Solution,Denmark,simplesolution.accelo.com,it-consulting,Marie Barker,2070.0,Zero,Kenneth,Sejberg,,,
1,Valentina Schichl,valentina.schichl@kaiser-x.com,2,NaN,2/21/2020,professional,199938,Kaiser X Labs,Germany,kaiserxlabs.accelo.com,it-consulting,Marie Barker,4182.0,2,Valentina,Schichl,,,
2,Velton Davis,velton@prospergroupcorp.com,3,"Very slow, convoluted, several errors.",2/19/2020,professional,175217,The Prosper Group,United States,prosper-group-corp.accelo.com,digital-marketing,Marie Barker,2530.0,3,Velton,Davis,,,
3,Francesca McMaster,fmcmaster@extent.com.au,Zero,NaN,2/23/2020,professional,180972,Extent Heritage,Australia,extent.accelo.com,other,Amy Stewart,2450.0,Zero,Francesca,McMaster,,,
4,Plamen | Verve?,plamen@verve-design.co.uk,Zero,NaN,2/23/2020,professional,152975,Verve Graphic Design & Marketing,United Kingdom,verve.accelo.com,web-design,Clay Hagen,759.0,Zero,Plamen,Verve,,,


In [295]:
# Output the result to CSV
data.to_csv('CSV Result/CLEANED.csv', index=False)